In [ ]:
import pandas as pd
import requests, uuid

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [ ]:
df = pd.read_csv("Hesas Misr_coments.csv")

df = df[df["message"].notnull()]

df.reset_index(drop=True, inplace=True)

In [ ]:
translate_key = "2c8e1ed333e24e7497ed4b92bfa57380"
translate_endpoint = "https://api.cognitive.microsofttranslator.com/"

Text_key = "b6bdcbefc8894568bed37af2f6f31029"
Text_endpoint = "https://internship-final-analysos.cognitiveservices.azure.com/"

#### authoricate text analysis client

In [ ]:
def authenticate_text_client():
    
    global Text_key
    global Text_endpoint
    
    key = Text_key
    endpoint = Text_endpoint

    text_analytics_client = TextAnalyticsClient( endpoint=endpoint,  credential= AzureKeyCredential(key))
    return text_analytics_client

text_analysis_client = authenticate_text_client()

## Translate function
**This functiontakes sentment and translate it to Englist.**

if any error occured due to access key, we can update it manually

In [ ]:
def translate(sent):
    
    global translate_key
    global translate_endpoint
    
    try:
     
        # subscription key and endpoint
        subscription_key = translate_key
        endpoint = translate_endpoint   
        
        # select endpoint resource url url
        constructed_url = endpoint + '/translate'
        
        # set request parameters and headers
        params = {
            'api-version': '3.0',
            'to': 'en',
            'toScript': 'latn'}  
        
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Ocp-Apim-Subscription-Region': "eastus",
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())}
        
        # set body {text or document to translate}.
        body = [{
            'text': sent}]
        
        # send the request and get data
        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()
        
        # print the result and return it
        print(response[0]['translations'][0]['text'])
        return response[0]['translations'][0]['text']
    
    except ConnectionError:
        print("do you want to update the key? y/n")
        ans = input()
        if ans == "y":
            print("enter new translate key>>   ")
            translate_key = input()
        else:
            pass

### NER function

In [ ]:
def NER(text):
    global text_analysis_client
    
    ## add lemit to the comment or its spam
    if len(text) > 750:
        return True
    
    try:
        # listing the text 
        documents = [text]
        # send request with the text
        NER_res = text_analysis_client.recognize_entities(documents = documents)[0]

        try:
            # check if hte result is person return True
            # if any error its not human name
            entity = NER_res.entities[0]
            if entity.category == 'Person':
                return True
            else:
              return False
        except IndexError:
            return False
        
    except Exception as err:
        print("Encountered exception. {}".format(err))

#### create data dict

In [ ]:
data = {
        "text":[],
        "Translate":[],
        "is_human":[],
        "Sentiment":[]
   
        }

In [ ]:
def sentiment_analysis(file, text, data):
    global text_analysis_client
    
    ## check if the text is human name
    is_human = NER(text)
    
    ## add data to dict and wright it to file
    data["text"].append(text)
    data["is_human"].append(is_human)
    print("\nis human: {}\n".format(is_human))
    file.write("\nis human: {}\n".format(is_human))
    
    
    # if the text is not human 
    # translate the text 
    # analyse text score
    
    if not(is_human):
        
        # translate the text/append the thanslation to dict
        translated_sent = translate(text)
        data["Translate"].append(translated_sent)

        
        ## analyse translated text score
        response =  text_analysis_client.analyze_sentiment(documents=[translated_sent])[0]
        ## P1 is the overall sentment 
        p1 = "Document Sentiment: {}".format(response.sentiment)
        ## P2 is every rate score 
        p2 = "\nOverall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            response.confidence_scores.positive,
            response.confidence_scores.neutral,
            response.confidence_scores.negative)
        
        ## print teh score
        print(p1)
        print(p2)
        
        ##write/append score to the file/dict
        file.write("Translate: {}\n".format(sent))
        file.write(p1+"\n")
        file.write(p2)
        data["Sentiment"].append("{}\n{}".format(p1, p2))

    
    # if the result is human name
    # will not ranslate and assume the text is neutral
    else:
        data["Translate"].append("not")
        data["Sentiment"].append("""Document Sentiment: neutral
                    Overall scores: positive=0.00; neutral=1.00; negative=0.00""")

### starting function

the function recalll itself untill the start number is the end number

> start from number to end number
>
> write tothe file and recall itself intil the start == end then return to the first 

In [ ]:
def start(file, start, end):
    global data
    if start == end:
        return
    try:
        text = df.message[start]
        file.write('{} \n'.format(text))
        
        sentiment_analysis(file, text, data)
        
        file.write("\n done {}\n".format(start))
        start(file, start+1, end)
        
    except KeyError as e:
        
       print("finished, {}".format(e))

### open file ands ave data

In [ ]:
start = int("")
end   = int("")

with open('{}{}-{}.txt'.format("Hessas",start,end-1),  'w', encoding='utf-8') as f:
    
    ## open start function
    start(f, k-1000)
    
    # after exit fromstart function 
    # save the data to .csv file and .txt  file
    df = pd.DataFrame(data)
    df.to_csv("{}{}-{}.csv".format("Hessas", start, end-1),index = False)